## Rasterising and combining the coastal and glacial shapefiles

Aim is to rasterise the shapefiles into the same projection so that they can then be combined into a single file. It is assumed that there is habitat for the birds if there is land and no ice coverage.

In [1]:
import os
import numpy as np
from pycoalescence import Map
import matplotlib.pyplot as plt

In [2]:
# Define our input data
original_glacial = os.path.join("data", "rasters", "original", "0.01MYA_deBoer_global_1deg_icethickness_compressed.tif")
coastline_shapefile = os.path.join("data", "shapefiles", "0.01MYA_-31m_FINAL.shp")
# Define the output data
coastline_raster = os.path.join("data", "rasters", "0.01MYA_-31m_FINAL.tif")
density_raster = os.path.join("data", "rasters", "density_0.01mya.tif")

In [3]:
# Get the geotransform and projection of the glacial data (which we will try to match with the coastlines)
m_glacial = Map(original_glacial)
target_geotransform = m_glacial.get_geo_transform()
target_proj = m_glacial.get_projection()

In [4]:
# Create the raster of the coastlines based on our target projection and geotransform
for file in [coastline_raster, density_raster]:
    if os.path.exists(file):
        os.remove(file)
m_coastline = Map()
m_coastline.rasterise(shape_file=coastline_shapefile, raster_file=coastline_raster, output_srs=target_proj,
                      geo_transform=target_geotransform, x_buffer=0, y_buffer=0, width=360, height=180)
# Create a new raster for density at 0.01 mya
m_coastline.create_copy(dst_file=density_raster)
m_density = Map(density_raster)
# Open the data into a numpy array
m_density.open()
# Remove any land area that is covered by ice. 
# The map has a lot of bands so see if any bands have a value > 0. 
for i in range(1, m_glacial.get_band_number() + 1):
    # Open a specific band number
    m_glacial.open(band_no=i)
    # Set any values in our density map to 0 where the glacial data is > 0
    m_density.data[np.ma.masked_where(m_glacial.data > 0, m_density.data).mask] = 0
# Write the data to the density map raster
m_density.write()

In [5]:
# Now show the new density plot
m_density.plot()
plt.show()